### Importing libraries

In [1]:
import openravepy
import trajoptpy
import json
import numpy as np
import trajoptpy.kin_utils as ku
from trajoptpy.check_traj import traj_is_safe
import time
import random 
import matplotlib.pyplot as plt
import pickle
from trajopt_util import *
from planning_util import *
from regression import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

FILENAME = 'data/dual_arm_fixed_init/'

### Setting the trajopt

In [2]:
np.set_printoptions(precision=4, suppress=True)

env = openravepy.Environment()
env.StopSimulation()
env.Load("robots/pr2-beta-static.zae")
env.Load("../env/bookshelves.env.xml")
env.SetDefaultViewer()

True

### Setting the robot

In [3]:
robot = env.GetRobots()[0]
left_arm = robot.GetManipulator("leftarm")
right_arm = robot.GetManipulator("rightarm")

robot.SetActiveDOFs(np.r_[right_arm.GetArmIndices(), left_arm.GetArmIndices()])
coeffs = [1,1,1,1,1,1,1,  1,1,1,1,1,1,1,] #coefficients for joint velocity cost

dof = len(robot.GetActiveDOFValues())
left_dof_away = np.array([ 0.5646,  1.1371, -0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
right_dof_away = np.array([ -0.5646,  1.1371, 0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
robot.SetDOFValues(left_dof_away, left_arm.GetArmIndices())
robot.SetDOFValues(right_dof_away, right_arm.GetArmIndices())
init_joint0 = robot.GetActiveDOFValues()

### Set the IK Solver 

In [4]:
hmat_target = np.array([[ 0.9689,  0.    , -0.2474,  0.4918],
       [-0.    ,  1.    ,  0.    ,  0.912 ],
       [ 0.2474, -0.    ,  0.9689,  0.8947],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])
quat_target = openravepy.quatFromRotationMatrix(hmat_target).tolist()
xyz_target = hmat_target[0:3,3].tolist()

# BEGIN IK for all manips
manips = [left_arm,right_arm]
for manip in manips:
    robot.SetActiveManipulator(manip)
    ikmodel = openravepy.databases.inversekinematics.InverseKinematicsModel(
        robot, iktype=openravepy.IkParameterization.Type.Transform6D)
    if not ikmodel.load():
        ikmodel.autogenerate()   
    
target_joint = ku.ik_for_link(hmat_target, left_arm, "l_gripper_tool_frame",
    filter_options = openravepy.IkFilterOptions.CheckEnvCollisions)
# END ik

### Construct a random problem 

### Build / Retrieve Dataset

In [5]:
def build_database(x_init, y_init, num_traj, limits, n_steps = 30, predictor=None):
    num_plan = 0
    comp_times = []
    costs = []

    tic = time.time()
    while len(x_init) < num_traj:
        print('{}th Planning, {} trajectories in database'.format(num_plan,len(x_init)))
        #Get a random pose as target
        target_joint_right,xyz_target_right,target_joint_left, \
                            xyz_target_left,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(target_joint)
        
        #Build memory using a set of predictors
        for func in predictor:
            #Predict the warmstart
            x_cur = np.concatenate([target_joint])[None,:]
            if isinstance(func, Straight_Regressor):
                traj,_ = func.predict(init_joint0, target_joint)     
            else:
                traj,_ = func.predict(x_cur)
                
            if func.is_transform == 'PCA':
                traj = func.pca.inverse_transform(traj)
            
            traj = traj.reshape(-1,dof)
            traj[0,:] = init_joint0
            
            #Plan
            robot.SetActiveDOFValues(init_joint0)
            request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
            request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
            request_traj = set_init(request_traj, 'given_traj', traj)
            duration, result = run_opt(request_traj, env)    

            #Check traj result
            traj = result.GetTraj()
            if check_traj(env,result, target_joint):
                print 'Planning is successfull!'
                x_init.append(np.concatenate(x_cur))
                y_init.append(traj.flatten())
                comp_times.append(duration)
                costs.append(result.GetCosts()[0][1])
                break
            else:
                print('Fail to find good solution!')
                continue        

        num_plan += 1
        
    
    #Store the result
    toc = time.time()
    total_time = toc-tic
    success_rate = num_traj*1.0/num_plan
    x_init = np.vstack(x_init)
    y_init = np.vstack(y_init)
    
    data = dict()
    data['x'] = x_init
    data['y'] = y_init
    data['total_time'] = total_time
    data['success_rate'] = success_rate
    data['comp_times'] = comp_times
    data['costs'] = costs
    return data

In [ ]:
to_build_database = True
to_augment_database = False
num_traj = 500
n_steps = 30
limits = dict()
limits[0] = [[0.47, 0.8],[-0.5, 0.5],[0.3,1.4]]

#build using straight-line warm-start
straight = Straight_Regressor(dof = dof, n_steps = n_steps)

if to_build_database:
    if to_augment_database:
        #load data
        data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
        x_init = list(data['x'])
        y_init = list(data['y'])
        num_traj += len(x_init)     
    else:
        x_init,y_init = [],[]
        
    data = build_database(x_init, y_init, num_traj,limits, n_steps = n_steps, predictor=[straight])
    x_init = data['x']
    y_init = data['y']
    pickle.dump(data,open(FILENAME + 'data.pkl', 'wb') )
    print('Success_rate : {}, average costs:{}'.format(data['success_rate'], np.mean(data['costs'])))
else:
    #load data
    data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
    x_init = data['x']
    y_init = data['y']
    num_traj = len(x_init)
    print('Obtain {} trajectories'.format(num_traj))

### Reduce the data sample

### Apply Dimensionality Reduction to the data

In [17]:
from sklearn.decomposition import PCA
y_pca = PCA(n_components = 50)
y_pca.fit(y_init)
y_init_reduced = y_pca.transform(y_init)

### Check Database 

### Build Function Approximator

In [ ]:
dim_input = len(x_init[0])
is_load_regressor = False

straight = Straight_Regressor(dof = dof, n_steps=n_steps)
nn = NN_Regressor()
gpy = GPy_Regressor(dim_input = dim_input)
gpy_pca = GPy_Regressor(dim_input = dim_input, is_transform='PCA')
gpy_pca.pca = y_pca
bgmr = DP_GLM_Regressor()
bgmr_pca = DP_GLM_Regressor(is_transform='PCA')
bgmr_pca.pca = y_pca
if is_load_regressor:
    nn.load_from_file(FILENAME + 'nn')
    gpy.load_from_file(FILENAME + 'gpy')
    gpy_pca.load_from_file(FILENAME + 'gpy_pca')
    bgmr.load_from_file(FILENAME + 'bgmr')
    bgmr_pca.load_from_file(FILENAME + 'bgmr_pca')
else:
    print 'Planning for NN'
    nn.fit(x_init, y_init)
    nn.save_to_file(FILENAME + 'nn')

    print 'Planning for GPY'
    gpy.fit(x_init, y_init, num_restarts=10)
    gpy.save_to_file(FILENAME + 'gpy')
    
    print 'Planning for GPY PCA'
    gpy_pca.fit(x_init, y_init_reduced)
    gpy_pca.save_to_file(FILENAME + 'gpy_pca')

    print 'Planning for bgmr'
    bgmr.fit(x_init,y_init)
    bgmr.save_to_file(FILENAME + 'bgmr')
    
    print 'Planning for bgmr PCA'
    bgmr_pca.fit(x_init,y_init_reduced)
    bgmr_pca.save_to_file(FILENAME + 'bgmr_pca')

### Creating the test cases

In [ ]:
create_test = True

if create_test:
    x_test = []
    target_joints = []
    num_test = 500
    for i in range(num_test):
        target_joint_right,xyz_target_right,target_joint_left,xyz_target_left,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(target_joint)
        cur_x = target_joint
        x_test.append(cur_x)

    x_test = np.vstack(x_test)
    data_test = dict()
    data_test['x_test'] = x_test
    test_file = open(FILENAME + 'data_test.pkl', 'wb')
    pickle.dump(data_test,test_file)
    test_file.close()
else:
    #load data
    test_file = open(FILENAME + 'data_test.pkl', 'rb')
    data_test = pickle.load(test_file)
    x_test = data_test['x_test']
    num_test = len(x_test)
    test_file.close()

### Testing the warmstart

In [ ]:
method_names = ['STD    ', 'NN     ', 'GPR    ', 'GPR_PCA', 'BGMR', 'BGMR_PCA']
methods = [straight, nn, gpy, gpy_pca,  bgmr, bgmr_pca]
results = dict()
for method in method_names:
    results[method] = dict()
    results[method]['costs'] = []
    results[method]['successes'] = []
    results[method]['comp_times'] = []
    results[method]['func_evals'] = []
    results[method]['qp_solves'] = []
    

num_test = 250
result,success, duration = dict(),dict(),dict()

for it in range(num_test):
    print('{}th Planning'.format(it))
    
    #setting up the problem case
    #index = np.random.randint(0,len(x_test))
    index = it
    x_cur = x_test[index:index+1,:]
    target_joint = x_cur.copy().flatten()
    robot.SetActiveDOFValues(init_joint0)
    
    #plan with various warmstarts
    request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
    request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
    
    for i,method in enumerate(methods):
        method_name = method_names[i]
        if isinstance(method, Straight_Regressor):
            traj,cov = method.predict(init_joint0, target_joint)
        else:
            traj,cov = method.predict(x_cur)
            
        if method.is_transform == 'PCA':
            traj = method.pca.inverse_transform(traj)
            
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint0
        robot.SetActiveDOFValues(init_joint0)
        request_traj = set_init(request_traj, 'given_traj', traj)
        duration[method_name], result[method_name] = run_opt(request_traj, env)    
        success[method_name] = check_traj(env,result[method_name], target_joint)

    #Record the result
    for method_name in method_names:
        results[method_name]['costs'].append(result[method_name].GetCosts()[0][1])
        results[method_name]['func_evals'].append(result[method_name].GetNumFuncEval()[0])
        results[method_name]['qp_solves'].append(result[method_name].GetNumQPSolve()[0])
        results[method_name]['successes'].append(success[method_name])      
        results[method_name]['comp_times'].append(duration[method_name])
        
    print_result(results,method_names)

### Printing Results

### Save Result

In [21]:
pickle.dump(results,open(FILENAME + 'result3.pkl', 'wb') )

### Load Result 

In [ ]:
results = pickle.load(open(FILENAME + 'result.pkl', 'rb'))